# phase 3: wav2vec2 fine-tuning for pd classification

fine-tune wav2vec2-base-960h on parkinson's disease voice detection using
leave-one-subject-out (loso) cross-validation for rigorous evaluation.

**methodology:**
- loso cv: same protocol as clinical baseline (88.3% accuracy) for fair comparison
- freeze cnn feature extractor + first 4 transformer layers (small dataset)
- gradient checkpointing for memory efficiency
- early stopping to prevent overfitting

**expected results:**
- target accuracy: 80-90% (competitive with clinical baseline)
- comparison with 17-feature clinical model establishes deep learning value

**hardware support:**
- nvidia gpu (cuda) - recommended
- apple silicon (mps) - supported but slower
- cpu - not recommended (10-20+ hours)

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. setup and configuration

In [2]:
import sys
from pathlib import Path
import os

# set project root
project_root = Path('/content/drive/MyDrive/pd-interpretability')

# Ensure the project root directory exists
if not project_root.exists():
    print(f"Project root not found: {project_root}. Creating directory...")
    project_root.mkdir(parents=True, exist_ok=True)
    print(f"Created directory: {project_root}")

os.chdir(project_root)
sys.path.insert(0, str(project_root))

print(f"project root: {project_root}")
print(f"working directory: {os.getcwd()}")

project root: /content/drive/MyDrive/pd-interpretability
working directory: /content/drive/.shortcut-targets-by-id/1abPvoWlTNsqv6oksf7mu8bJcJD6F9bKg/pd-interpretability


In [3]:
import shutil

repo_url = "https://github.com/smayan-gowda/pd-interpretability"
repo_name = repo_url.split('/')[-1]

# Assuming os.chdir(project_root) was successful in the previous cell
# Check if the current directory (project_root) contains a .git repository
if (project_root / ".git").exists():
    print(f"Repository already exists directly at {project_root}. Pulling latest changes...")
    # Ensure we are in the correct directory
    os.chdir(project_root)
    !git pull
    print(f"Working directory: {os.getcwd()}")
else:
    # Check if there is a nested repo and try to fix it
    nested_repo_candidate = project_root / repo_name
    if nested_repo_candidate.is_dir() and (nested_repo_candidate / ".git").exists():
        print(f"Found existing nested repository at {nested_repo_candidate}. Moving contents up to {project_root}...")
        for item in nested_repo_candidate.iterdir():
            # Avoid moving the nested folder into itself if it's accidentally named the same
            if item.name == nested_repo_candidate.name and item.is_dir():
                continue
            shutil.move(str(item), str(project_root / item.name))
        shutil.rmtree(nested_repo_candidate)
        print(f"Moved contents and removed nested directory.")

        # After moving, the .git directory should now be in project_root, so re-evaluate
        if (project_root / ".git").exists():
            print(f"Repository now established directly at {project_root}. Pulling latest changes...")
            os.chdir(project_root)
            !git pull
            print(f"Working directory: {os.getcwd()}")
        else:
            # If after moving, .git is still not there, then clone directly
            print(f"Cloning repository {repo_name} directly into {project_root} after fixing nested structure...")
            os.chdir(project_root) # Ensure we are in project_root
            !git clone {repo_url} .
            print(f"Working directory: {os.getcwd()}")
    else:
        # No .git in project_root and no fixable nested repo, so clone directly
        print(f"Cloning repository {repo_name} directly into {project_root}...")
        os.chdir(project_root) # Ensure we are in project_root
        !git clone {repo_url} .
        print(f"Working directory: {os.getcwd()}")

# Update sys.path to include the repository root
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# project_root is already correctly set by the Path object, no need to re-assign from repo_path
print(f"project root (after git operations): {project_root}")
print(f"working directory (after git operations): {os.getcwd()}")

Repository already exists directly at /content/drive/MyDrive/pd-interpretability. Pulling latest changes...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 12 (delta 8), reused 10 (delta 6), pack-reused 0 (from 0)
Unpacking objects: 100% (12/12), 1.93 KiB | 0 bytes/s, done.
From https://github.com/smayan-gowda/pd-interpretability
   9fbd1ff..480f521  main       -> origin/main
Updating 50e067b..480f521
error: Your local changes to the following files would be overwritten by merge:
	notebooks/colab/03_train_wav2vec2.ipynb
Please commit your changes or stash them before you merge.
Aborting
Working directory: /content/drive/.shortcut-targets-by-id/1abPvoWlTNsqv6oksf7mu8bJcJD6F9bKg/pd-interpretability
project root (after git operations): /content/drive/MyDrive/pd-interpretability
working directory (after git operations): /content/drive/.shortcut-targets-by-id/1abPvoWlTNsqv6oksf7mu8bJcJD6F9bKg/p

In [4]:
import torch
import numpy as np
import pandas as pd
import json
import warnings
from datetime import datetime
from tqdm import tqdm
from typing import Dict, List, Tuple, Optional
from collections import defaultdict
!pip install torchcodec

warnings.filterwarnings('ignore')

def detect_device():
    """detect best available compute device."""
    if torch.cuda.is_available():
        device = 'cuda'
        device_name = torch.cuda.get_device_name(0)
        memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"detected: nvidia gpu ({device_name})")
        print(f"vram: {memory_gb:.1f} gb")
        return device, True
    elif torch.backends.mps.is_available():
        device = 'mps'
        print("detected: apple silicon (mps)")
        return device, True
    else:
        device = 'cpu'
        print("warning: no gpu detected, using cpu (very slow)")
        return device, False

device, has_accelerator = detect_device()
print(f"pytorch version: {torch.__version__}")
print(f"device: {device}")

detected: nvidia gpu (Tesla T4)
vram: 15.8 gb
pytorch version: 2.9.0+cu126
device: cuda


In [5]:
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from torch.optim import AdamW

from transformers import (
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2FeatureExtractor,
    get_linear_schedule_with_warmup
)

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report
)

from src.data.datasets import ItalianPVSDataset
from src.models.classifier import DataCollatorWithPadding

print("imports complete")

imports complete


In [6]:
# experiment configuration - optimized for stable training on small medical datasets
# key fixes: lower learning rate, gradient clipping, more frozen layers, label smoothing

config = {
    # model
    'model_name': 'facebook/wav2vec2-base-960h',
    'num_labels': 2,
    'freeze_feature_extractor': True,
    'freeze_encoder_layers': 8,  # freeze first 8 of 12 transformer layers (critical for small datasets)
    'hidden_dropout': 0.1,
    'attention_dropout': 0.1,
    'final_dropout': 0.3,  # higher for classification head (regularization)
    'layerdrop': 0.0,  # CRITICAL: disable layerdrop for training stability

    # audio
    'max_duration': 10.0,
    'target_sr': 16000,

    # training - conservative settings for small medical dataset
    'num_epochs': 20,  # more epochs with slower learning
    'learning_rate': 2e-5,  # CRITICAL: reduced from 5e-5 to prevent NaN
    'warmup_ratio': 0.2,  # increased warmup for stability
    'weight_decay': 0.01,
    'max_grad_norm': 1.0,  # CRITICAL: gradient clipping to prevent exploding gradients
    'label_smoothing': 0.1,  # regularization for small dataset
    'early_stopping_patience': 5,  # increased patience with lower lr

    # loso cv
    'max_folds': 3,  # set to 3 for quick test, None for full CV

    # random seed
    'random_seed': 42
}

# device-specific settings
if device == 'cuda':
    config['batch_size'] = 8
    config['gradient_accumulation_steps'] = 4
    # CRITICAL: fp16 disabled - Wav2Vec2 produces NaN in fp16 on small datasets
    # The attention mechanism overflows in half precision with this data
    config['fp16'] = False
    config['gradient_checkpointing'] = True
elif device == 'mps':
    config['batch_size'] = 4
    config['gradient_accumulation_steps'] = 8
    config['fp16'] = False  # mps fp16 unstable
    config['gradient_checkpointing'] = False  # causes memory leak on mps
else:
    config['batch_size'] = 2
    config['gradient_accumulation_steps'] = 16
    config['fp16'] = False
    config['gradient_checkpointing'] = False

effective_batch = config['batch_size'] * config['gradient_accumulation_steps']
print(f"batch size: {config['batch_size']} (effective: {effective_batch})")
print(f"learning rate: {config['learning_rate']}")
print(f"max gradient norm: {config['max_grad_norm']}")
print(f"label smoothing: {config['label_smoothing']}")
print(f"epochs: {config['num_epochs']}")
print(f"max folds: {config['max_folds']} (set to None for full LOSO CV)")
print(f"frozen layers: cnn + first {config['freeze_encoder_layers']} of 12 transformer layers")
print(f"layerdrop: {config['layerdrop']} (disabled for stability)")
print(f"fp16: {config['fp16']} (disabled to prevent NaN in Wav2Vec2 attention)")

batch size: 8 (effective: 32)
learning rate: 2e-05
max gradient norm: 1.0
label smoothing: 0.1
epochs: 20
max folds: 3 (set to None for full LOSO CV)
frozen layers: cnn + first 8 of 12 transformer layers
layerdrop: 0.0 (disabled for stability)
fp16: False (disabled to prevent NaN in Wav2Vec2 attention)


## 2. load dataset

In [7]:
# load dataset with caching enabled to avoid slow disk i/o during training
data_root = project_root / 'data' / 'raw'

print("loading dataset with audio caching enabled...")
print("this will pre-load all audio into ram to avoid disk reads during training")

dataset = ItalianPVSDataset(
    root_dir=str(data_root / 'italian_pvs'),
    task=None,  # all tasks
    target_sr=config['target_sr'],
    max_duration=config['max_duration'],
    cache_audio=True  # critical: cache audio in memory to avoid i/o bottleneck
)

print(f"samples: {len(dataset)}")

# extract labels and subject ids for loso cv
labels = np.array([s['label'] for s in dataset.samples])
subject_ids = np.array([s['subject_id'] for s in dataset.samples])

# unique subjects
unique_subjects = np.unique(subject_ids)
n_subjects = len(unique_subjects)

# class distribution
n_pd = np.sum(labels)
n_hc = len(labels) - n_pd
print(f"class distribution: {n_hc} hc, {n_pd} pd")
print(f"subjects: {n_subjects}")
print(f"loso cv folds: {n_subjects}")

loading dataset with audio caching enabled...
this will pre-load all audio into ram to avoid disk reads during training
samples: 831
class distribution: 394 hc, 437 pd
subjects: 61
loso cv folds: 61


In [8]:
# pre-cache all audio data into ram before training
# this is critical for performance

print("=" * 60)
print("PRE-CACHING ALL AUDIO INTO MEMORY")
print("=" * 60)
print(f"samples to cache: {len(dataset)}")
print()

cache_start = datetime.now()
failed_samples = []

for i in tqdm(range(len(dataset)), desc="caching audio", unit="sample"):
    try:
        # accessing each sample triggers caching
        _ = dataset[i]
    except Exception as e:
        failed_samples.append((i, str(e)))

cache_time = (datetime.now() - cache_start).total_seconds()

print(f"\ncaching complete in {cache_time:.1f}s ({cache_time/60:.1f}m)")
print(f"cached samples: {len(dataset) - len(failed_samples)}/{len(dataset)}")

if failed_samples:
    print(f"failed samples: {len(failed_samples)}")
    for idx, err in failed_samples[:5]:
        print(f"  sample {idx}: {err}")

# verify cache is populated
cache_size = len(dataset._audio_cache) if dataset._audio_cache else 0
estimated_memory_mb = cache_size * config['max_duration'] * config['target_sr'] * 4 / (1024 * 1024)
print(f"cache size: {cache_size} samples (~{estimated_memory_mb:.0f} mb estimated)")
print("=" * 60)

PRE-CACHING ALL AUDIO INTO MEMORY
samples to cache: 831



caching audio: 100%|██████████| 831/831 [00:40<00:00, 20.45sample/s]


caching complete in 40.7s (0.7m)
cached samples: 831/831
cache size: 831 samples (~507 mb estimated)


In [9]:
# create output directory
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
experiment_name = f"wav2vec2_loso_{timestamp}"
output_dir = project_root / 'results' / 'checkpoints' / experiment_name
output_dir.mkdir(parents=True, exist_ok=True)

# save config
config_path = output_dir / 'config.json'
with open(config_path, 'w') as f:
    json.dump(config, f, indent=2)

print(f"experiment: {experiment_name}")
print(f"output: {output_dir}")

experiment: wav2vec2_loso_20260103_192930
output: /content/drive/MyDrive/pd-interpretability/results/checkpoints/wav2vec2_loso_20260103_192930


## 3. training utilities

In [ ]:
def create_model(config: dict, device: str):
    """create fresh wav2vec2 model with optimized freezing strategy for small datasets."""
    from transformers import Wav2Vec2Config

    # create config with stability settings
    model_config = Wav2Vec2Config.from_pretrained(
        config['model_name'],
        num_labels=config['num_labels'],
        hidden_dropout=config.get('hidden_dropout', 0.1),
        attention_dropout=config.get('attention_dropout', 0.1),
        final_dropout=config.get('final_dropout', 0.3),
        layerdrop=config.get('layerdrop', 0.0),  # CRITICAL: disable for stability
        classifier_proj_size=256,
    )

    model = Wav2Vec2ForSequenceClassification.from_pretrained(
        config['model_name'],
        config=model_config,
        ignore_mismatched_sizes=True
    )

    # enable gradient checkpointing
    if config.get('gradient_checkpointing', False):
        model.gradient_checkpointing_enable()

    # freeze cnn feature extractor (ALWAYS for wav2vec2)
    if config.get('freeze_feature_extractor', True):
        model.freeze_feature_encoder()

    # freeze first n transformer layers (freeze MORE for small datasets)
    freeze_layers = config.get('freeze_encoder_layers', 8)
    if freeze_layers > 0:
        for i, layer in enumerate(model.wav2vec2.encoder.layers):
            if i < freeze_layers:
                for param in layer.parameters():
                    param.requires_grad = False

    # count parameters
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen_params = total_params - trainable_params

    print(f"      model parameters: {total_params:,} total, {trainable_params:,} trainable ({100*trainable_params/total_params:.1f}%), {frozen_params:,} frozen")

    return model.to(device)


def count_parameters(model):
    """count trainable and frozen parameters."""
    total = sum(p.numel() for p in model.parameters())
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total, trainable, total - trainable


def create_collate_fn(feature_extractor, max_length: int):
    """
    create collate function for wav2vec2 training.
    
    CRITICAL: wav2vec2 expects RAW waveforms, NOT normalized.
    the model's internal feature extractor handles all preprocessing.
    manual normalization causes NaN loss!
    """
    def collate_fn(batch: List[Dict]) -> Dict[str, torch.Tensor]:
        # extract raw audio waveforms - dataset returns 1d tensors
        input_values = [item['input_values'] for item in batch]
        labels = torch.tensor([item['label'] for item in batch], dtype=torch.long)

        # pad sequences to max length in batch
        max_len = min(max(len(x) for x in input_values), max_length)

        padded_input = torch.zeros(len(input_values), max_len)
        attention_mask = torch.zeros(len(input_values), max_len)

        for i, wav in enumerate(input_values):
            length = min(len(wav), max_len)
            # CRITICAL: pass raw audio directly, NO normalization!
            # wav2vec2 expects raw waveforms in natural amplitude range
            padded_input[i, :length] = wav[:length]
            attention_mask[i, :length] = 1.0

        return {
            'input_values': padded_input,
            'attention_mask': attention_mask,
            'labels': labels
        }

    return collate_fn


def compute_class_weights(labels: np.ndarray) -> torch.Tensor:
    """compute inverse frequency class weights for imbalanced data."""
    from collections import Counter
    counts = Counter(labels)
    total = len(labels)
    # inverse frequency weighting
    weights = torch.tensor([total / counts[0], total / counts[1]], dtype=torch.float32)
    # normalize so weights sum to num_classes
    weights = weights / weights.sum() * len(counts)
    return weights

In [11]:
import gc
import time
import math

def train_epoch(model, loader, optimizer, scheduler, scaler, device, config, epoch_num=None, verbose=True):
    """
    train for one epoch with gradient accumulation, clipping, and NaN detection.

    key stability features:
    - gradient clipping (max_grad_norm)
    - NaN/Inf detection and recovery
    - label smoothing loss
    - proper memory management
    """
    model.train()
    total_loss = 0
    n_batches = 0
    nan_batches = 0
    optimizer.zero_grad()

    accumulation_steps = config.get('gradient_accumulation_steps', 4)
    max_grad_norm = config.get('max_grad_norm', 1.0)
    label_smoothing = config.get('label_smoothing', 0.1)

    # create label smoothing loss
    loss_fn = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    # create progress bar
    pbar = tqdm(loader, desc=f"epoch {epoch_num}" if epoch_num else "training",
                leave=False, unit="batch") if verbose else loader

    batch_times = []

    for step, batch in enumerate(pbar):
        batch_start = time.time()

        input_values = batch['input_values'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # check for NaN in inputs (silently skip)
        if torch.isnan(input_values).any() or torch.isinf(input_values).any():
            del input_values, attention_mask, labels
            nan_batches += 1
            continue

        try:
            if scaler is not None:
                with torch.cuda.amp.autocast():
                    outputs = model(input_values, attention_mask=attention_mask)
                    loss = loss_fn(outputs.logits, labels) / accumulation_steps

                # check for NaN loss (silently skip)
                if torch.isnan(loss) or torch.isinf(loss):
                    del input_values, attention_mask, labels, outputs, loss
                    optimizer.zero_grad()
                    nan_batches += 1
                    continue

                scaler.scale(loss).backward()
            else:
                outputs = model(input_values, attention_mask=attention_mask)
                loss = loss_fn(outputs.logits, labels) / accumulation_steps

                # check for NaN loss (silently skip)
                if torch.isnan(loss) or torch.isinf(loss):
                    del input_values, attention_mask, labels, outputs, loss
                    optimizer.zero_grad()
                    nan_batches += 1
                    continue

                loss.backward()

            total_loss += loss.item() * accumulation_steps
            n_batches += 1

            if (step + 1) % accumulation_steps == 0:
                if scaler is not None:
                    scaler.unscale_(optimizer)

                # gradient clipping (CRITICAL for stability)
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)

                if scaler is not None:
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    optimizer.step()

                scheduler.step()
                optimizer.zero_grad()

        except RuntimeError as e:
            optimizer.zero_grad()
            nan_batches += 1
            continue

        # memory management - synchronize periodically
        if device == 'mps' and (step + 1) % 25 == 0:
            torch.mps.synchronize()
        elif device == 'cuda' and (step + 1) % 100 == 0:
            torch.cuda.synchronize()

        # cleanup batch tensors
        del input_values, attention_mask, labels, outputs, loss

        batch_time = time.time() - batch_start
        batch_times.append(batch_time)

        # update progress bar with current stats
        if verbose and hasattr(pbar, 'set_postfix'):
            avg_batch_time = sum(batch_times[-10:]) / len(batch_times[-10:])
            current_loss = total_loss / n_batches if n_batches > 0 else float('nan')
            pbar.set_postfix({
                'loss': f'{current_loss:.4f}',
                'nan': nan_batches
            }, refresh=False)

    # end of epoch cleanup
    if device == 'mps':
        torch.mps.synchronize()
        torch.mps.empty_cache()
        gc.collect()
    elif device == 'cuda':
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

    avg_loss = total_loss / n_batches if n_batches > 0 else float('nan')

    # only print warning at end of epoch if there were NaN batches
    if nan_batches > 0 and nan_batches < len(loader):
        print(f"    note: {nan_batches}/{len(loader)} batches skipped (NaN)")

    return avg_loss

In [12]:
@torch.no_grad()
def evaluate(model, loader, device, config=None, desc="evaluating"):
    """evaluate model on dataset with memory management and progress tracking."""
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []

    # use label smoothing in eval too for consistency
    label_smoothing = config.get('label_smoothing', 0.0) if config else 0.0
    loss_fn = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

    pbar = tqdm(loader, desc=desc, leave=False, unit="batch")

    for batch in pbar:
        input_values = batch['input_values'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_values, attention_mask=attention_mask)
        loss = loss_fn(outputs.logits, labels)
        total_loss += loss.item()

        probs = torch.softmax(outputs.logits, dim=-1)
        preds = outputs.logits.argmax(dim=-1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())

        # cleanup batch tensors
        del input_values, attention_mask, labels, outputs, probs, preds, loss

    # memory cleanup
    if device == 'mps':
        torch.mps.synchronize()
        torch.mps.empty_cache()
        gc.collect()
    elif device == 'cuda':
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

    n_batches = len(loader)

    return {
        'loss': total_loss / n_batches if n_batches > 0 else 0,
        'accuracy': accuracy_score(all_labels, all_preds),
        'predictions': np.array(all_preds),
        'labels': np.array(all_labels),
        'probabilities': np.array(all_probs)
    }

## 4. loso cross-validation training

In [ ]:
def train_fold(
    dataset,
    train_indices: np.ndarray,
    test_indices: np.ndarray,
    config: dict,
    device: str,
    fold_idx: int,
    output_dir: Path
) -> Dict:
    """
    train model on single loso fold with comprehensive stability measures.

    key improvements:
    - no manual normalization (wav2vec2 handles it internally)
    - gradient clipping
    - label smoothing
    - NaN detection and recovery
    - class weight handling for imbalanced data
    - proper learning rate scheduling
    """

    fold_start_time = time.time()

    print(f"    [fold {fold_idx + 1}] creating data subsets...")
    # create data subsets
    train_subset = Subset(dataset, train_indices.tolist())
    test_subset = Subset(dataset, test_indices.tolist())

    # compute class weights for training data
    train_labels = np.array([dataset.samples[i]['label'] for i in train_indices])
    class_weights = compute_class_weights(train_labels)
    print(f"    [fold {fold_idx + 1}] class weights: HC={class_weights[0]:.3f}, PD={class_weights[1]:.3f}")

    # feature extractor and custom collate function
    # CRITICAL: do_normalize=False because we pass raw audio to wav2vec2
    feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
        config['model_name'],
        return_attention_mask=False
    )
    max_length = int(config['max_duration'] * config['target_sr'])
    collate_fn = create_collate_fn(feature_extractor, max_length)

    print(f"    [fold {fold_idx + 1}] creating dataloaders (batch_size={config['batch_size']})...")
    # dataloaders - num_workers=0 for cached data to avoid pickle overhead
    train_loader = DataLoader(
        train_subset,
        batch_size=config['batch_size'],
        shuffle=True,
        collate_fn=collate_fn,
        num_workers=0,  # use 0 for cached data
        pin_memory=(device == 'cuda'),
        drop_last=True  # avoid small batches that can cause instability
    )

    test_loader = DataLoader(
        test_subset,
        batch_size=config['batch_size'] * 2,
        shuffle=False,
        collate_fn=collate_fn,
        num_workers=0,
        pin_memory=(device == 'cuda')
    )

    print(f"    [fold {fold_idx + 1}] initializing model ({config['model_name']})...")
    model_init_start = time.time()
    # create fresh model with stability settings
    model = create_model(config, device)
    print(f"    [fold {fold_idx + 1}] model initialized in {time.time() - model_init_start:.1f}s")

    # optimizer with numerical stability settings
    optimizer = AdamW(
        [p for p in model.parameters() if p.requires_grad],
        lr=config['learning_rate'],
        weight_decay=config['weight_decay'],
        eps=1e-8  # numerical stability
    )

    # scheduler with warmup for stability
    steps_per_epoch = max(1, len(train_loader) // config['gradient_accumulation_steps'])
    total_steps = steps_per_epoch * config['num_epochs']
    warmup_steps = int(total_steps * config['warmup_ratio'])

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    # gradient scaler for fp16 (disabled in config for stability)
    if config.get('fp16', False) and device == 'cuda':
        scaler = torch.cuda.amp.GradScaler(
            init_scale=2**10,
            growth_interval=100
        )
    else:
        scaler = None

    print(f"    [fold {fold_idx + 1}] starting training ({config['num_epochs']} epochs, {len(train_loader)} batches/epoch)...")
    print(f"    [fold {fold_idx + 1}] lr={config['learning_rate']}, warmup={warmup_steps}/{total_steps} steps")

    # training loop with early stopping
    best_loss = float('inf')
    best_acc = 0.0
    patience_counter = 0
    best_metrics = None
    epoch_times = []

    for epoch in range(config['num_epochs']):
        epoch_start = time.time()

        train_loss = train_epoch(
            model, train_loader, optimizer, scheduler, scaler,
            device, config,
            epoch_num=epoch + 1, verbose=True
        )

        # check for training failure
        if math.isnan(train_loss):
            print(f"      WARNING: training loss is NaN at epoch {epoch + 1}")
            print(f"      this should NOT happen with fixed collate function - check audio data!")
            break

        test_metrics = evaluate(model, test_loader, device, config, desc=f"eval epoch {epoch+1}")

        epoch_time = time.time() - epoch_start
        epoch_times.append(epoch_time)

        current_lr = optimizer.param_groups[0]['lr']
        print(f"      epoch {epoch + 1}/{config['num_epochs']}: "
              f"train_loss={train_loss:.4f}, test_loss={test_metrics['loss']:.4f}, "
              f"test_acc={test_metrics['accuracy']:.1%}, lr={current_lr:.2e}, time={epoch_time:.1f}s")

        # save best metrics (by validation loss)
        if test_metrics['loss'] < best_loss:
            best_loss = test_metrics['loss']
            best_acc = test_metrics['accuracy']
            patience_counter = 0
            best_metrics = test_metrics.copy()
        else:
            patience_counter += 1
            if patience_counter >= config['early_stopping_patience']:
                print(f"      early stopping triggered at epoch {epoch + 1}")
                break

        # cleanup after each epoch
        if device == 'cuda':
            torch.cuda.synchronize()
            torch.cuda.empty_cache()
        elif device == 'mps':
            torch.mps.synchronize()
            torch.mps.empty_cache()
        gc.collect()

    print(f"    [fold {fold_idx + 1}] final evaluation...")
    # final evaluation
    final_metrics = evaluate(model, test_loader, device, config, desc="final eval") if best_metrics is None else best_metrics

    fold_total_time = time.time() - fold_start_time
    print(f"    [fold {fold_idx + 1}] completed in {fold_total_time:.1f}s ({fold_total_time/60:.1f}m)")
    print(f"    [fold {fold_idx + 1}] best accuracy: {best_acc:.1%}")

    print(f"    [fold {fold_idx + 1}] cleaning up...")
    # aggressive cleanup
    del model, optimizer, scheduler, train_loader, test_loader
    del train_subset, test_subset, feature_extractor, collate_fn

    if device == 'cuda':
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    elif device == 'mps':
        torch.mps.synchronize()
        torch.mps.empty_cache()

    gc.collect()

    return {
        'fold': fold_idx,
        'train_samples': len(train_indices),
        'test_samples': len(test_indices),
        'accuracy': final_metrics['accuracy'],
        'predictions': final_metrics['predictions'],
        'labels': final_metrics['labels'],
        'probabilities': final_metrics['probabilities'],
        'fold_time': fold_total_time,
        'avg_epoch_time': sum(epoch_times) / len(epoch_times) if epoch_times else 0,
        'best_loss': best_loss
    }

In [ ]:
# run loso cross-validation
logo = LeaveOneGroupOut()
n_folds = logo.get_n_splits(groups=subject_ids)

if config['max_folds']:
    n_folds = min(n_folds, config['max_folds'])

print("=" * 80)
print(f"STARTING LOSO CROSS-VALIDATION")
print("=" * 80)
print(f"total folds to run: {n_folds}")
print(f"device: {device}")
print(f"model: {config['model_name']}")
print(f"batch size: {config['batch_size']}")
print(f"learning rate: {config['learning_rate']}")
print(f"max epochs per fold: {config['num_epochs']}")
print(f"early stopping patience: {config['early_stopping_patience']}")
print("=" * 80)

fold_results = []
all_predictions = []
all_labels = []
all_probabilities = []
all_subject_ids = []

start_time = datetime.now()

for fold_idx, (train_idx, test_idx) in enumerate(
    logo.split(np.arange(len(dataset)), labels, subject_ids)
):
    if config['max_folds'] and fold_idx >= config['max_folds']:
        break

    fold_start = datetime.now()

    test_subject = subject_ids[test_idx[0]]
    test_label = labels[test_idx[0]]
    label_str = "pd" if test_label == 1 else "hc"

    print(f"\n{'=' * 80}")
    print(f"FOLD {fold_idx + 1}/{n_folds}")
    print(f"{'=' * 80}")
    print(f"  test subject: {test_subject} ({label_str})")
    print(f"  train samples: {len(train_idx)}, test samples: {len(test_idx)}")
    print(f"  starting training...")

    result = train_fold(
        dataset=dataset,
        train_indices=train_idx,
        test_indices=test_idx,
        config=config,
        device=device,
        fold_idx=fold_idx,
        output_dir=output_dir
    )

    fold_time = (datetime.now() - fold_start).total_seconds()
    elapsed_total = datetime.now() - start_time
    avg_time_per_fold = elapsed_total.total_seconds() / (fold_idx + 1)
    remaining_folds = n_folds - (fold_idx + 1)
    eta = remaining_folds * avg_time_per_fold

    fold_results.append(result)
    all_predictions.extend(result['predictions'])
    all_labels.extend(result['labels'])
    all_probabilities.extend(result['probabilities'])
    all_subject_ids.extend([test_subject] * len(result['predictions']))

    # calculate running accuracy
    running_acc = accuracy_score(all_labels, all_predictions)

    print(f"\n  FOLD {fold_idx + 1} COMPLETE:")
    print(f"    fold accuracy: {result['accuracy']:.1%}")
    print(f"    fold time: {fold_time:.1f}s ({fold_time/60:.1f}m)")
    print(f"    running overall accuracy: {running_acc:.1%}")
    print(f"    time elapsed: {elapsed_total}")
    print(f"    estimated time remaining: {eta/60:.1f}m ({eta/3600:.1f}h)")
    print(f"{'=' * 80}")

elapsed = datetime.now() - start_time
print(f"\n{'=' * 80}")
print(f"LOSO CV COMPLETE")
print(f"{'=' * 80}")
print(f"total time: {elapsed} ({elapsed.total_seconds()/60:.1f}m)")
print(f"{'=' * 80}")

STARTING LOSO CROSS-VALIDATION
total folds to run: 3
device: cuda
model: facebook/wav2vec2-base-960h
batch size: 8
learning rate: 2e-05
max epochs per fold: 20
early stopping patience: 5

FOLD 1/3
  test subject: HC_elderly_AGNESE_P (hc)
  train samples: 815, test samples: 16
  starting training...
    [fold 1] creating data subsets...
    [fold 1] class weights: HC=1.072, PD=0.928
    [fold 1] creating dataloaders (batch_size=8)...
    [fold 1] initializing model (facebook/wav2vec2-base-960h)...


Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


      model parameters: 94,569,090 total, 33,665,666 trainable (35.6%), 60,903,424 frozen
    [fold 1] model initialized in 0.9s
    [fold 1] starting training (20 epochs, 101 batches/epoch)...
    [fold 1] lr=2e-05, warmup=100/500 steps


      attempting to continue with reduced learning rate...


      attempting to continue with reduced learning rate...


...

      attempting to continue with reduced learning rate...


      attempting to continue with reduced learning rate...


      attempting to continue with reduced learning rate...


      attempting to continue with reduced learning rate...


epoch 8:  24%|██▍       | 24/101 [00:08<00:28,  2.74batch/s]

## 5. aggregate results

In [ ]:
# convert to numpy arrays
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)
all_probabilities = np.array(all_probabilities)

# overall metrics
overall_accuracy = accuracy_score(all_labels, all_predictions)
overall_precision = precision_score(all_labels, all_predictions, zero_division=0)
overall_recall = recall_score(all_labels, all_predictions, zero_division=0)
overall_f1 = f1_score(all_labels, all_predictions, zero_division=0)

try:
    overall_auc = roc_auc_score(all_labels, all_probabilities)
except:
    overall_auc = 0.5

# per-fold accuracy
fold_accuracies = [r['accuracy'] for r in fold_results]
mean_accuracy = np.mean(fold_accuracies)
std_accuracy = np.std(fold_accuracies)

# confusion matrix
cm = confusion_matrix(all_labels, all_predictions)

print("=" * 60)
print("LOSO CROSS-VALIDATION RESULTS")
print("=" * 60)
print(f"\noverall metrics (aggregated across all folds):")
print(f"  accuracy: {overall_accuracy:.1%}")
print(f"  precision: {overall_precision:.3f}")
print(f"  recall: {overall_recall:.3f}")
print(f"  f1 score: {overall_f1:.3f}")
print(f"  auc-roc: {overall_auc:.3f}")

print(f"\nper-fold statistics:")
print(f"  mean accuracy: {mean_accuracy:.1%} ± {std_accuracy:.1%}")
print(f"  min: {min(fold_accuracies):.1%}, max: {max(fold_accuracies):.1%}")

print(f"\nconfusion matrix:")
print(f"           predicted")
print(f"            hc    pd")
print(f"actual hc  {cm[0,0]:4d}  {cm[0,1]:4d}")
print(f"       pd  {cm[1,0]:4d}  {cm[1,1]:4d}")

In [ ]:
# per-subject accuracy analysis
subject_results = {}
for subj, pred, label in zip(all_subject_ids, all_predictions, all_labels):
    if subj not in subject_results:
        subject_results[subj] = {'correct': 0, 'total': 0, 'label': label}
    subject_results[subj]['total'] += 1
    if pred == label:
        subject_results[subj]['correct'] += 1

subject_accuracies = []
subject_data = []

for subj, data in subject_results.items():
    acc = data['correct'] / data['total']
    subject_accuracies.append(acc)
    diagnosis = 'pd' if data['label'] == 1 else 'hc'
    subject_data.append({
        'subject_id': subj,
        'diagnosis': diagnosis,
        'accuracy': acc,
        'correct': data['correct'],
        'total': data['total']
    })

subject_df = pd.DataFrame(subject_data)
subject_df = subject_df.sort_values('accuracy', ascending=False)

print(f"\nper-subject accuracy:")
print(f"  mean: {np.mean(subject_accuracies):.1%}")
print(f"  median: {np.median(subject_accuracies):.1%}")
print(f"  subjects with 100% accuracy: {sum(1 for a in subject_accuracies if a == 1.0)}/{len(subject_accuracies)}")
print(f"  subjects with <50% accuracy: {sum(1 for a in subject_accuracies if a < 0.5)}/{len(subject_accuracies)}")

## 6. comparison with clinical baseline

In [ ]:
# load clinical baseline results for comparison
baseline_path = project_root / 'results' / 'clinical_baseline_results.json'

if baseline_path.exists():
    with open(baseline_path) as f:
        baseline_results = json.load(f)

    clinical_acc = baseline_results['svm']['accuracy_mean']
    clinical_std = baseline_results['svm']['accuracy_std']

    print("=" * 60)
    print("COMPARISON WITH CLINICAL BASELINE")
    print("=" * 60)
    print(f"\nclinical baseline (svm, 17 features):")
    print(f"  accuracy: {clinical_acc:.1%} \u00b1 {clinical_std:.1%}")

    print(f"\nwav2vec2 (loso cv):")
    print(f"  accuracy: {overall_accuracy:.1%}")
    print(f"  per-fold mean: {mean_accuracy:.1%} \u00b1 {std_accuracy:.1%}")

    diff = overall_accuracy - clinical_acc
    print(f"\ndifference: {diff:+.1%}")

    if diff > 0:
        print("  wav2vec2 outperforms clinical baseline")
    elif diff < -0.05:
        print("  clinical baseline outperforms wav2vec2")
    else:
        print("  comparable performance")
else:
    print("clinical baseline results not found")

## 7. save results

In [ ]:
# save comprehensive results
results = {
    'experiment': experiment_name,
    'config': config,
    'device': device,
    'n_folds': len(fold_results),
    'n_subjects': n_subjects,
    'n_samples': len(dataset),

    'overall_metrics': {
        'accuracy': float(overall_accuracy),
        'precision': float(overall_precision),
        'recall': float(overall_recall),
        'f1': float(overall_f1),
        'auc': float(overall_auc)
    },

    'per_fold_stats': {
        'mean_accuracy': float(mean_accuracy),
        'std_accuracy': float(std_accuracy),
        'min_accuracy': float(min(fold_accuracies)),
        'max_accuracy': float(max(fold_accuracies))
    },

    'confusion_matrix': cm.tolist(),

    'fold_results': [
        {k: v.tolist() if isinstance(v, np.ndarray) else v for k, v in r.items()}
        for r in fold_results
    ],

    'timestamp': datetime.now().isoformat(),
    'elapsed_time': str(elapsed)
}

# save to json
results_path = output_dir / 'wav2vec2_loso_results.json'
with open(results_path, 'w') as f:
    json.dump(results, f, indent=2)

# save subject-level results
subject_path = output_dir / 'wav2vec2_subject_accuracy.csv'
subject_df.to_csv(subject_path, index=False)

# also save to main results folder for easy access
main_results_path = project_root / 'results' / 'wav2vec2_loso_results.json'
with open(main_results_path, 'w') as f:
    json.dump(results, f, indent=2)

print(f"results saved to:")
print(f"  {results_path}")
print(f"  {subject_path}")
print(f"  {main_results_path}")

## 8. visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# publication-quality style with latex and times new roman
plt.rcParams.update({
    'text.usetex': False, # Changed from True to False
    #'text.latex.preamble': r'\usepackage{amsmath}\usepackage{amssymb}', # Removed as not needed when usetex is False
    'font.family': 'serif',
    'font.serif': ['Times', 'Times New Roman', 'DejaVu Serif'],
    'font.size': 10,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'axes.linewidth': 0.8,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'xtick.direction': 'out',
    'ytick.direction': 'out',
    'legend.fontsize': 9,
    'legend.frameon': True,
    'legend.fancybox': False,
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1,
})

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# fold accuracy distribution
ax1 = axes[0]
ax1.bar(range(1, len(fold_accuracies) + 1), fold_accuracies, color='#3498db', alpha=0.8, edgecolor='white')
ax1.axhline(y=mean_accuracy, color='#e74c3c', linestyle='--', linewidth=2,
            label=rf'Mean: {mean_accuracy:.1%}')
ax1.set_xlabel(r'Fold')
ax1.set_ylabel(r'Accuracy')
ax1.set_title(r'Per-Fold Accuracy (LOSO CV)')
ax1.legend(loc='lower right')
ax1.set_ylim(0, 1.05)

# confusion matrix
ax2 = axes[1]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax2,
            xticklabels=[r'HC', r'PD'], yticklabels=[r'HC', r'PD'],
            cbar_kws={'label': 'Count'})
ax2.set_xlabel(r'Predicted')
ax2.set_ylabel(r'Actual')
ax2.set_title(rf'Confusion Matrix (Accuracy: {overall_accuracy:.1%})')

# subject accuracy histogram
ax3 = axes[2]
ax3.hist(subject_accuracies, bins=10, color='#3498db', alpha=0.8, edgecolor='white')
ax3.axvline(x=np.mean(subject_accuracies), color='#e74c3c', linestyle='--', linewidth=2,
            label=rf'Mean: {np.mean(subject_accuracies):.1%}')
ax3.set_xlabel(r'Accuracy')
ax3.set_ylabel(r'Count')
ax3.set_title(r'Per-Subject Accuracy Distribution')
ax3.legend()

plt.tight_layout()

# save to experiment dir
plt.savefig(output_dir / 'wav2vec2_results_summary.png', dpi=300, bbox_inches='tight')
plt.savefig(output_dir / 'wav2vec2_results_summary.pdf', dpi=300, bbox_inches='tight')

# also save to main figures folder
main_fig_dir = project_root / 'results' / 'figures'
main_fig_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(main_fig_dir / 'fig_p3_01_wav2vec2_loso_summary.png', dpi=300, bbox_inches='tight')
plt.savefig(main_fig_dir / 'fig_p3_01_wav2vec2_loso_summary.pdf', dpi=300, bbox_inches='tight')

plt.show()

print(f"figures saved to {output_dir} and {main_fig_dir}")

In [ ]:
# Install cm-super for additional LaTeX fonts, which often includes type1ec.sty
!apt-get install -y cm-super

# Update TeX Live filename database and font maps to recognize new packages
!texhash
!updmap-sys --force
!fmtutil-sys --all

In [ ]:
# Install TeX Live packages for LaTeX rendering in matplotlib
!apt-get update
!apt-get install -y texlive-latex-extra texlive-fonts-recommended dvipng

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# publication-quality style with latex and times new roman
plt.rcParams.update({
    'text.usetex': True, # Revert to True to use LaTeX
    'text.latex.preamble': r'\usepackage{amsmath}\usepackage{amssymb}', # Re-add LaTeX preamble
    'font.family': 'serif',
    'font.serif': ['Times', 'Times New Roman', 'DejaVu Serif'],
    'font.size': 10,
    'axes.titlesize': 11,
    'axes.labelsize': 10,
    'axes.linewidth': 0.8,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'xtick.direction': 'out',
    'ytick.direction': 'out',
    'legend.fontsize': 9,
    'legend.frameon': True,
    'legend.fancybox': False,
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1,
})

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# fold accuracy distribution
ax1 = axes[0]
ax1.bar(range(1, len(fold_accuracies) + 1), fold_accuracies, color='#3498db', alpha=0.8, edgecolor='white')
ax1.axhline(y=mean_accuracy, color='#e74c3c', linestyle='--', linewidth=2,
            label=rf'Mean: {mean_accuracy:.1%}')
ax1.set_xlabel(r'Fold')
ax1.set_ylabel(r'Accuracy')
ax1.set_title(r'Per-Fold Accuracy (LOSO CV)')
ax1.legend(loc='lower right')
ax1.set_ylim(0, 1.05)

# confusion matrix
ax2 = axes[1]
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax2,
            xticklabels=[r'HC', r'PD'], yticklabels=[r'HC', r'PD'],
            cbar_kws={'label': 'Count'})
ax2.set_xlabel(r'Predicted')
ax2.set_ylabel(r'Actual')
ax2.set_title(rf'Confusion Matrix (Accuracy: {overall_accuracy:.1%})')

# subject accuracy histogram
ax3 = axes[2]
ax3.hist(subject_accuracies, bins=10, color='#3498db', alpha=0.8, edgecolor='white')
ax3.axvline(x=np.mean(subject_accuracies), color='#e74c3c', linestyle='--', linewidth=2,
            label=rf'Mean: {np.mean(subject_accuracies):.1%}')
ax3.set_xlabel(r'Accuracy')
ax3.set_ylabel(r'Count')
ax3.set_title(r'Per-Subject Accuracy Distribution')
ax3.legend()

plt.tight_layout()

# save to experiment dir
plt.savefig(output_dir / 'wav2vec2_results_summary.png', dpi=300, bbox_inches='tight')
plt.savefig(output_dir / 'wav2vec2_results_summary.pdf', dpi=300, bbox_inches='tight')

# also save to main figures folder
main_fig_dir = project_root / 'results' / 'figures'
main_fig_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(main_fig_dir / 'fig_p3_01_wav2vec2_loso_summary.png', dpi=300, bbox_inches='tight')
plt.savefig(main_fig_dir / 'fig_p3_01_wav2vec2_loso_summary.pdf', dpi=300, bbox_inches='tight')

plt.show()

print(f"figures saved to {output_dir} and {main_fig_dir}")

In [ ]:
# comparison bar chart with clinical baseline
if baseline_path.exists():
    fig, ax = plt.subplots(figsize=(8, 5))

    models = [r'Clinical Baseline' + '\n' + r'(SVM, 17 features)',
              r'Wav2Vec2' + '\n' + r'(Fine-tuned)']
    accuracies = [clinical_acc, overall_accuracy]
    stds = [clinical_std, std_accuracy]
    colors = ['#2ecc71', '#3498db']

    bars = ax.bar(models, accuracies, yerr=stds, capsize=8, color=colors, alpha=0.8,
                  edgecolor='white', linewidth=1.5)

    for bar, acc, std in zip(bars, accuracies, stds):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.02,
                rf'{acc:.1%}', ha='center', va='bottom', fontsize=11, fontweight='bold')

    ax.set_ylabel(r'Accuracy')
    ax.set_title(r'Model Comparison: LOSO Cross-Validation')
    ax.set_ylim(0, 1.15)
    ax.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5, label=r'Chance Level')
    ax.legend(loc='upper right')

    plt.tight_layout()

    # save figures
    plt.savefig(output_dir / 'model_comparison.png', dpi=300, bbox_inches='tight')
    plt.savefig(output_dir / 'model_comparison.pdf', dpi=300, bbox_inches='tight')

    main_fig_dir = project_root / 'results' / 'figures'
    plt.savefig(main_fig_dir / 'fig_p3_02_model_comparison.png', dpi=300, bbox_inches='tight')
    plt.savefig(main_fig_dir / 'fig_p3_02_model_comparison.pdf', dpi=300, bbox_inches='tight')

    plt.show()

    print(f"comparison figures saved")
else:
    print("clinical baseline results not found - skipping comparison")

## 9. summary and next steps

In [ ]:
print("=" * 60)
print("PHASE 3 COMPLETE: WAV2VEC2 FINE-TUNING")
print("=" * 60)

print(f"\nmodel: {config['model_name']}")
print(f"device: {device}")
print(f"loso cv folds: {len(fold_results)}")
print(f"training time: {elapsed}")

print(f"\nresults:")
print(f"  accuracy: {overall_accuracy:.1%}")
print(f"  precision: {overall_precision:.3f}")
print(f"  recall: {overall_recall:.3f}")
print(f"  f1 score: {overall_f1:.3f}")
print(f"  auc-roc: {overall_auc:.3f}")

print(f"\nper-fold accuracy: {mean_accuracy:.1%} ± {std_accuracy:.1%}")

print(f"\nnext steps:")
print(f"  1. phase 4: activation extraction (notebook 04)")
print(f"  2. phase 5: probing experiments (notebook 05)")
print(f"  3. phase 6: activation patching (notebook 06)")

print(f"\noutputs saved to: {output_dir}")

## 10. train final model for activation extraction

train a single model on all data for use in probing and patching experiments.
this model will be used to extract activations in phase 4.

In [ ]:
# train final model on 80% of data (hold out 20% for testing)
# uses the same corrected approach as LOSO CV training

print("=" * 60)
print("TRAINING FINAL MODEL FOR ACTIVATION EXTRACTION")
print("=" * 60)

train_subset, _, test_subset = dataset.get_subject_split(
    test_size=0.2,
    val_size=0.0,
    random_state=config['random_seed']
)

print(f"training final model...")
print(f"  train samples: {len(train_subset)}")
print(f"  test samples: {len(test_subset)}")

# compute class weights
train_indices = train_subset.indices
train_labels = np.array([dataset.samples[i]['label'] for i in train_indices])
class_weights = compute_class_weights(train_labels)
print(f"  class weights: HC={class_weights[0]:.3f}, PD={class_weights[1]:.3f}")

# create model with stability settings
final_model = create_model(config, device)

# feature extractor - no normalization, wav2vec2 handles it internally
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(
    config['model_name'],
    return_attention_mask=False
)
max_length = int(config['max_duration'] * config['target_sr'])
collate_fn = create_collate_fn(feature_extractor, max_length)

# dataloaders
train_loader = DataLoader(
    train_subset,
    batch_size=config['batch_size'],
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=(device == 'cuda'),
    drop_last=True
)

test_loader = DataLoader(
    test_subset,
    batch_size=config['batch_size'] * 2,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=(device == 'cuda')
)

# optimizer and scheduler
optimizer = AdamW(
    [p for p in final_model.parameters() if p.requires_grad],
    lr=config['learning_rate'],
    weight_decay=config['weight_decay'],
    eps=1e-8
)

steps_per_epoch = max(1, len(train_loader) // config['gradient_accumulation_steps'])
total_steps = steps_per_epoch * config['num_epochs']
warmup_steps = int(total_steps * config['warmup_ratio'])

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=total_steps
)

# gradient scaler for fp16 (disabled in config for stability)
if config.get('fp16', False) and device == 'cuda':
    scaler = torch.cuda.amp.GradScaler(
        init_scale=2**10,
        growth_interval=100
    )
else:
    scaler = None

print(f"  epochs: {config['num_epochs']}")
print(f"  learning rate: {config['learning_rate']}")
print(f"  warmup steps: {warmup_steps}/{total_steps}")
print()

# training loop with early stopping
best_acc = 0
best_loss = float('inf')
patience_counter = 0
checkpoint_path = None

for epoch in range(config['num_epochs']):
    train_loss = train_epoch(
        final_model, train_loader, optimizer, scheduler, scaler,
        device, config, epoch_num=epoch + 1, verbose=True
    )

    # check for NaN
    if math.isnan(train_loss):
        print(f"  WARNING: NaN loss at epoch {epoch + 1}")
        print(f"  this should NOT happen with fixed collate function - check audio data!")
        break

    test_metrics = evaluate(final_model, test_loader, device, config)

    current_lr = optimizer.param_groups[0]['lr']
    print(f"  epoch {epoch+1}/{config['num_epochs']}: loss={train_loss:.4f}, "
          f"test_loss={test_metrics['loss']:.4f}, acc={test_metrics['accuracy']:.1%}, lr={current_lr:.2e}")

    # save best model
    if test_metrics['accuracy'] > best_acc:
        best_acc = test_metrics['accuracy']
        best_loss = test_metrics['loss']
        patience_counter = 0
        # save checkpoint
        checkpoint_path = output_dir / 'final_model'
        checkpoint_path.mkdir(parents=True, exist_ok=True)
        final_model.save_pretrained(checkpoint_path)
        feature_extractor.save_pretrained(checkpoint_path)
        print(f"    -> saved best model (acc: {best_acc:.1%})")
    else:
        patience_counter += 1
        if patience_counter >= config['early_stopping_patience']:
            print(f"  early stopping at epoch {epoch + 1}")
            break

    # cleanup
    if device == 'cuda':
        torch.cuda.empty_cache()
    gc.collect()

print()
print("=" * 60)
print("FINAL MODEL TRAINING COMPLETE")
print("=" * 60)
print(f"  best test accuracy: {best_acc:.1%}")
print(f"  best test loss: {best_loss:.4f}")
if checkpoint_path:
    print(f"  model saved to: {checkpoint_path}")

    # also copy to main results folder for easy access
    main_model_path = project_root / 'results' / 'final_model'
    main_model_path.mkdir(parents=True, exist_ok=True)
    final_model.save_pretrained(main_model_path)
    feature_extractor.save_pretrained(main_model_path)
    print(f"  also saved to: {main_model_path}")
print("=" * 60)